# Задание:

Основной мотив исследования такой: скоро сезон спадет и выручки в аптеке начнут падать. У ребят есть механизм СМС-рассылок, но они не хотят тратить бюджеты впустую - они хотят провести персонализированную рассылку.

Идея такая: нет смысла предлагать персональную скидку для повышения среднего чека тем, кто вообще покупал один раз и то аскорбинку.

- Соответственно, цели исследования:

1. Провести классификацию пользователей
2. Сформировать рекомендации - какой группе пользователей что предлагать

Как выбирались метрики.

Для задачи персонализированной рассылки ключевая цель — понять:

* кто покупает часто,
* кто покупает много,
* кто покупает недавно,
* кто приносит высокую выручку,
* кого имеет смысл стимулировать скидками.

## Метрики 
* RFM
* ABC
* Перцентили

## Почему именно эти метрики

1. RFM — международный стандарт персонализации.
Он прост, легко объясним бизнесу, хорошо масштабируется и идеален для SMS-кампаний.

2. RFM отражает поведение клиента полностью:

* R → актуальность

* F → лояльность

* M → ценность

3. ABC-товаров позволяет связать клиентов с товарами.
Можно понять, какие категории интересны ядру, а какие — «разовым».

4. Выбор порогов через перцентили — метод, не зависящий от сезона и не требующий экспертных догадок.

### Создание RFM-таблицы по картам пользователей

Этот запрос рассчитывает RFM-метрики (Recency, Frequency, Monetary) для каждой карты лояльности и присваивает баллы от 1 до 3 по каждой метрике.  
В результате получаем полную RFM-таблицу — основу для дальнейшей сегментации клиентов.

#### SQL-запрос (PostgreSQL)

```sql
WITH base AS (
    SELECT
        card
      , (DATE '2022-06-09' - MAX(dr_dat::date)) AS recency_days
      , COUNT(DISTINCT dr_ndoc) AS frequency
      , SUM(dr_kol * dr_croz - COALESCE(dr_sdisc, 0)) AS monetary
    FROM (
        SELECT
            s.*
          , b.card
        FROM sales s
        JOIN bonuscheques b 
          ON s.dr_ndoc = split_part(b.doc_id, '#', 2)::int
        WHERE s.dr_dat BETWEEN '2022-05-01' AND '2022-06-09'
    ) t
    GROUP BY card
)
, thresholds AS (
    SELECT
        percentile_cont(0.33) WITHIN GROUP (ORDER BY recency_days) AS r_p33
      , percentile_cont(0.66) WITHIN GROUP (ORDER BY recency_days) AS r_p66
      , percentile_cont(0.33) WITHIN GROUP (ORDER BY frequency)     AS f_p33
      , percentile_cont(0.66) WITHIN GROUP (ORDER BY frequency)     AS f_p66
      , percentile_cont(0.33) WITHIN GROUP (ORDER BY monetary)     AS m_p33
      , percentile_cont(0.66) WITHIN GROUP (ORDER BY monetary)     AS m_p66
    FROM base
)
SELECT
    b.card
  , CASE
        WHEN b.recency_days <= t.r_p33 THEN 3
        WHEN b.recency_days <= t.r_p66 THEN 2
        ELSE 1
    END AS r_group
  , CASE
        WHEN b.frequency <= t.f_p33 THEN 1
        WHEN b.frequency <= t.f_p66 THEN 2
        ELSE 3
    END AS f_group
  , CASE
        WHEN b.monetary <= t.m_p33 THEN 1
        WHEN b.monetary <= t.m_p66 THEN 2
        ELSE 3
    END AS m_group
FROM base b
CROSS JOIN thresholds t
ORDER BY r_group DESC;

### Сегментация пользователей по RFM для SMS-рассылок

Цель — разделить клиентов на понятные сегменты, чтобы точнее планировать SMS-кампании:  
кому слать акции на возвращение, кому — благодарность, а кому — новинки.

#### SQL-запрос (PostgreSQL)

```sql
WITH base AS (
    SELECT
        card
      , (DATE '2022-06-09' - MAX(dr_dat::date)) AS recency_days
      , COUNT(DISTINCT dr_ndoc) AS frequency
      , SUM(dr_kol * dr_croz - COALESCE(dr_sdisc, 0)) AS monetary
    FROM (
        SELECT
            s.*
          , b.card
        FROM sales s
        JOIN bonuscheques b 
          ON s.dr_ndoc = split_part(b.doc_id, '#', 2)::int
        WHERE s.dr_dat BETWEEN '2022-05-01' AND '2022-06-09'
    ) t
    GROUP BY card
)
, thresholds AS (
    SELECT
        percentile_cont(0.33) WITHIN GROUP (ORDER BY recency_days) AS r_p33
      , percentile_cont(0.66) WITHIN GROUP (ORDER BY recency_days) AS r_p66
      , percentile_cont(0.33) WITHIN GROUP (ORDER BY frequency)     AS f_p33
      , percentile_cont(0.66) WITHIN GROUP (ORDER BY frequency)     AS f_p66
      , percentile_cont(0.33) WITHIN GROUP (ORDER BY monetary)     AS m_p33
      , percentile_cont(0.66) WITHIN GROUP (ORDER BY monetary)     AS m_p66
    FROM base
)
, rfm AS (
    SELECT
        b.card
      , CASE
            WHEN b.recency_days <= t.r_p33 THEN 3
            WHEN b.recency_days <= t.r_p66 THEN 2
            ELSE 1
        END AS r_group
      , CASE
            WHEN b.frequency <= t.f_p33 THEN 1
            WHEN b.frequency <= t.f_p66 THEN 2
            ELSE 3
        END AS f_group
      , CASE
            WHEN b.monetary <= t.m_p33 THEN 1
            WHEN b.monetary <= t.m_p66 THEN 2
            ELSE 3
        END AS m_group
    FROM base b
    CROSS JOIN thresholds t
)
SELECT
    CONCAT(r_group, f_group, m_group) AS rfm_code
  , COUNT(*) AS users_count
FROM rfm
GROUP BY rfm_code
ORDER BY rfm_code DESC;

# 🎯 Рекомендации по работе с RFM-сегментами

### 1★ **Высокий приоритет** — Спящие киты  
**Сегменты:** 133 · 132 · 131  
**Клиентов:** 130  
**Действие:** Персональная скидка **25–35%** только на товары с `profit_abc = 'A'`  
**Цель:** Максимальная прибыль даже со скидкой  

### 2★ Лояльные, но давно не были  
**Сегменты:** 233 · 232 · 313 · 331  
**Клиентов:** 341  
**Действие:**  
• 20–25% на топовые товары  
• Акция: купи 2 дорогих — третий в подарок (из категории A)  
**Цель:** Поднять чек, сохранить маржу  

### 3★ Средне-активные (реактивация)  
**Сегменты:** 212 · 232 · 312 · 213  
**Клиентов:** 556  
**Действие:**  
• 15–20% на маржинальные товары  
• Или 300 бонусов при покупке от 2000 ₽  
**Цель:** Бюджетное возвращение клиентов  

### 4★ **Чемпионы** — Лучшие клиенты  
**Сегменты:** 333 · 332 · 331  
**Клиентов:** 479  
**Действие:** **Без скидок!** Только:  
• 500–1000 бонусов  
• Ранний доступ к новинкам  
• VIP-статус  
**Цель:** Удержание без потери маржи  

### 5★ Недавние новички  
**Сегменты:** 311 · 231 · 211  
**Клиентов:** 554  
**Действие:**  
• 10–15% на популярные товары  
• Подарок при покупке от 1500 ₽  
**Цель:** Приучить к высоким чекам  

### 6★ Спящие слабые  
**Сегменты:** 113 · 112  
**Клиентов:** 416  
**Действие:** Мягкое напоминание (10% на всё) или не беспокоить  
**Цель:** Низкий ROI — минимальные затраты  

### 7★ Потерянные  
**Сегменты:** 111  
**Клиентов:** 359  
**Действие:** Не тратить бюджет  
**Цель:** 99% не вернутся  

*Обновлено: 14 декабря 2025*

### ABC-классификация товаров по выручке

ABC-анализ делит товары на три группы по доле в общей выручке:
- **A** — 80% выручки (самые важные товары)
- **B** — следующие 15% выручки
- **C** — остальные 5% (мало влияют)

#### SQL-запрос (PostgreSQL)

```sql
WITH agg_abc AS (
    SELECT
        dr_ndrugs
      , SUM(dr_kol * dr_croz - dr_sdisc) AS revenue
    FROM sales
    GROUP BY dr_ndrugs
)
, abc AS (
    SELECT
        dr_ndrugs AS product
      , revenue
      , CASE
            WHEN SUM(revenue) OVER (ORDER BY revenue DESC) /
                 SUM(revenue) OVER () * 100.0 <= 80 THEN 'A'
            WHEN SUM(revenue) OVER (ORDER BY revenue DESC) /
                 SUM(revenue) OVER () * 100.0 <= 95 THEN 'B'
            ELSE 'C'
        END AS revenue_abc
    FROM agg_abc
)
SELECT
    product
  , revenue
  , revenue_abc
FROM abc
ORDER BY revenue DESC;